# Sprawozdanie 2 z Uczenia maszynowego

### Jakub Zdanowski 127239

## 1. Zbiór danych

Postanowiłem użyć zbioru danych, który jest często stosowany we wprowadzeniu do uczenia maszynowego. [Dane](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) opierają się na danych spisowych ludności z Kalifornii z 1990 roku.

Poniżej zostaną przedstawione podstawowe informacje takie jak liczba przykładów, średnia, odchylenie standardowe, max, min i różne kwantyle.

In [12]:
import pandas as pd

# wczytanie danych z pliku .csv
california_housing_dataframe = pd.read_csv("california_housing_train.csv")

# ilustracja podstawowych informacji o danych
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [13]:
# Pokazuje 5 pierwszych wierszy w danych dla 
# zilustrowania typu i przykładowych wartości
california_housing_dataframe.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


### Co zawierają dane?
Wszystkie dane są typu *float64*.

Powyższa tabela ukazuje podstawowe (liczbowe) informacje o danych, poniżej krótki opis kazdej z kolumn:
- **longitude** - szerokość geograficzna
- **latitude** - długość geograficzna
- **housing_median_age** - średni wiek domu
- **total_rooms** - całkowita liczba pokoi w bloku
- **total_bedrooms** - całkowita liczba sypialni w bloku
- **population** - całkowita liczba osób mieszkających w bloku
- **households** - łączna liczba gospodarstw domowych (grupa osób mieszkających w jednostce domowej) na blok
- **median_income** - mediana dochodu dla gospodarstwa domowego w bloku (mierzona w dziesiątkach tysięcy dolarów amerykańskich)
- **median_house_value** - mediana wartości domu dla gospodarstw domowych w bloku (mierzona w dolarach amerykańskich)

### Wstępna analiza danych
Pierwszymi

## 2. Predykcja wartości domu

bla bla bla

In [53]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Dane zostaną przemieszane, aby mieć pewność, że nie pojawią się żadne patologiczne efekty porządkowania, które mogłyby zaszkodzić wydajności SGD. Dodatkowo, zmieniona zostanie wartość `median_house_value` na jednostki tysiąca, aby można było się jej nauczyć nieco łatwiej przy współczynnikach uczenia w niskim zakresie.

In [54]:
# przemieszanie kolejności danych
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))

# zmniejszenie zakresu wartości mieszkań
california_housing_dataframe["median_house_value"] /= 1000.0

In [55]:
# wybór kolumn do predykcji wartości mieszkania
X_feature = california_housing_dataframe[["total_rooms"]]

# opis danych funkcji; nie zawierają samych danych
X_column = [tf.feature_column.numeric_column("total_rooms")]

In [56]:
# definicja predykcji
y_target = california_housing_dataframe["median_house_value"]

In [57]:
# użycie spadku gradientu do optymalizacji 
linear_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)

# Powszechnie stosowany mechanizm zmniejszania problemu gradientu eksplodującego
# poprzez sztuczne ograniczanie (przycinanie) maksymalnej wartości gradientów
linear_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# konfiguracja regresora liniowego wraz z wybranymi kolumnami
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns = X_column,
    optimizer = linear_optimizer
)

`input_func` funkcja pozwalająca na użycie danych wczytanych przez *pandas* w estymatorach *tensorflow*. Napisana na podstawie [przewodnika tensorflow](https://www.tensorflow.org/guide/datasets).

In [58]:
# funckja użyta będzie w kolejnych zadaniach
def input_func(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    # konwertowanie do tablic numpy'owych
    features = {key:np.array(value) for key, value in dict(features).items()} 
    
    # Construct a dataset, and configure batching/repeating.
    # przy większych danych należy zwrócić uwagę czy zmieszczą się w pamięci
    ds = Dataset.from_tensor_slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
        
    # zwrócenie kolejnej paczki (batch) danych
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels    

In [59]:
_ = linear_regressor.train(
    input_fn = lambda:input_func(X_feature, y_target),
    steps=100
)

In [60]:
# Create an input function for predictions.
# Note: Since we're making just one prediction for each example, we don't 
# need to repeat or shuffle the data here.
prediction_input_fn =lambda: input_func(X_feature, y_target, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, y_target)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 0.000
Root Mean Squared Error (on training data): 0.001
